In [1]:
from sklearn.datasets import fetch_20newsgroups # data
from nltk.tokenize import word_tokenize
import re

In [2]:
def get_real_word_position(data):
    """
    data --> e.g. data = reuters.words('training/9865')
    """
    from nltk.corpus import wordnet
    
    leng = len(data)
    word_pos = []
    for i in range(len(data)):
        #if (not wordnet.synsets(data[i])) or str.isdigit(data[i]) or (len(data[i])<2) or data[i].isupper():
        if (not wordnet.synsets(data[i])) or str.isdigit(data[i]) or (len(data[i])<2):
            continue
        else:
            word_pos.append(i)
    return word_pos

In [3]:
def random_int_list(start, stop, length):

    import random

    start, stop = (int(start), int(stop)) if start <= stop else (int(stop), int(start))
    length = int(abs(length)) if length else 0
    random_list = []
    for i in range(length):
        random_list.append(random.randint(start, stop))
    return random_list

In [4]:
def get_masked_word_position(word_pos):
    """
    word_pos --> e.g. word_pos = get_real_word_position(data)
    """

    rd = random_int_list(0,len(word_pos)-1,2)
    
    cor_word_pos = []
    for i in rd:
        cor_word_pos.append(word_pos[i])
    
    cor_word_pos = list(set(cor_word_pos))
    cor_word_pos.sort()

    
    return cor_word_pos

In [5]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

model = BertForMaskedLM.from_pretrained('bert-large-uncased')
model.eval()
%%capture

UsageError: Line magic function `%%capture` not found.


In [ ]:
def get_masked_word_list(cor_word_pos,token_list):
    """
    cor_word_pos --> e.g. cor_word_pos = get_masked_word_position(word_pos)
    """
    cor_word=[]
    for i in cor_word_pos:
        cor_word.append(token_list[i])
    cor_word
    
    return cor_word

In [ ]:
def get_train_text(data, cor_word_pos):

    train_list = list(data)
    for i in cor_word_pos:
        train_list[i] = '[MASK]'
    train_text = '[CLS] ' + ' '.join(train_list) + ' [SEP]'
    
    return train_text

In [ ]:
def predict(text, correct_word_list, tokenizer, model):
    
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    masked_list = []
    for i in range(len(tokenized_text)):
        if tokenized_text[i] == "[MASK]":
            masked_list.append(i)
            
    segments_ids = [0] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
    
    predict_list=[]
    for i in masked_list:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        predict_list.append(predicted_token)
        
    print("----------------------------------------------")
    print("Predicted word | Original word: ")
    for i in range(len(correct_word_list)):
        print(predict_list[i]," | ",correct_word_list[i])
    print("Paragraph: ")
    print(text)
    print("----------------------------------------------")
        
    return predict_list

In [27]:
news_dataset = fetch_20newsgroups(subset='all',remove=('headers','footers','quotes'))#获取并缓存数据
documents = news_dataset.data

In [85]:
text_list, word_list = [],[]
for i in range(1000):    
    test = documents[i].replace('\n',' ')
    temp = word_tokenize(test)
    word_pos = get_real_word_position(temp)
    try:
        cor_word_pos = get_masked_word_position(word_pos)
    except:
        continue
    cor_word = get_masked_word_list(cor_word_pos,temp)
    train_text = get_train_text(temp,cor_word_pos)
    text_list.append(train_text)
    word_list.append(cor_word)

In [87]:
len(word_list)

971

In [88]:
dic = dict(zip(text_list,word_list))

In [7]:
import pickle
with open(r'D:\Fordham\DS Research\submission\20201110\text_dict.pickle', 'rb') as file:
    dic = pickle.load(file)

In [8]:
def model_test(text_dict,tokenizer, model):
    
    total_num = 0
    correct_num = 0
    for text, word in text_dict.items():
        total_num = total_num+len(word)
        try:
            predict_list = predict(text,word,tokenizer,model)
        except:
            continue
        for i in range(len(word)):
                if word[i].lower() == predict_list[i].lower():
                    correct_num+=1
                
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    print("The number of total masked word: ",total_num)
    print("The number of predicted word correctly: ",correct_num)
    print("Accuracy rate: ",correct_num/total_num)
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    
    return 

In [78]:
predict(text_list[1],word_list[1],tokenizer,model)

----------------------------------------------
Predicted word | Original word: 
other  |  email
email  |  other
Paragraph: 
[CLS] My brother is in the market for a high-performance video card that supports VESA local bus with 1-2MB RAM . Does anyone have suggestions/ideas on : - Diamond Stealth Pro Local Bus - Orchid Farenheit 1280 - ATI Graphics Ultra Pro - Any [MASK] high-performance VLB card Please post or [MASK] . Thank you ! - Matt [SEP]
----------------------------------------------


['other', 'email']

In [13]:
model_test(dic,tokenizer,model)

----------------------------------------------
Predicted word | Original word: 
lucky  |  disappointed
team  |  PENS
Paragraph: 
[CLS] I am sure some bashers of Pens fans are pretty confused about the lack of any kind of posts about the recent Pens massacre of the Devils . Actually , I am bit puzzled too and a bit relieved . However , I am going to put an end to non-PIttsburghers ' relief with a bit of praise for the Pens . Man , they are killing those Devils worse than I thought . Jagr just showed you why he is much better than his regular season stats . He is also a lot fo fun to watch in the playoffs . Bowman should let JAgr have a lot of fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway . I was very [MASK] not to see the Islanders lose the final regular season game . [MASK] RULE ! ! ! [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
ram  |  RAM
pro  |  Gr

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1580 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
has  |  has
take  |  take
Paragraph: 
[CLS] How about Kirlian imaging ? I believe the FAQ for sci.skeptics ( sp ? ) [MASK] a nice write-up on this . They would certainly be most supportive on helping you to build such a device and connect to a 120Kvolt supply so that you can [MASK] a serious look at your `` aura '' ... : - ) [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
is  |  is
no  |  no
Paragraph: 
[CLS] There [MASK] [MASK] notion of heliocentric , or even galacticentric either . -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
rejects  |  criticized
all  |  some
Paragraph: 
[CLS] In the following report : _Turkey Eyes

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (998 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
doubts  |  doubts
is  |  is
Paragraph: 
[CLS] ed > 1 . All of us that argue about gyroscopes , etc. , throughly understand ed > the technique of countersteering . me > Including all the ones who think that they countersteer all the way me > through a corner ? ? ed > Well ... all the way through a decreasing-radius corner , anyway ... Maybe they are riding around an ever-decreasing circle of lies which eventually leads to the truth ... . me > The official line here ( though I do have my [MASK] about it ) is that the me > front brake is applied first , followed by the rear brake , the idea being me > that you avoid locking up the rear after weight transfer takes place . Me too , though unfortunately the `` Official Line '' [MASK] the one that you have to adhere to if you want to get a full licence . The examiner 's guidelines are laid down by the government , and the basic rider education courses have no choi

...

----------------------------------------------
Predicted word | Original word: 
thing  |  one
by  |  by
Paragraph: 
[CLS] May I ask why they are afraid to do so ? -- - Speaking of proofs of God , the funniest [MASK] I have ever seen was in a term paper handed in by a freshman . She wrote , `` God must exist , because he would n't be so mean as to make me believe he exists if he really does n't ! '' Is this argument really so much worse than the ontological proofs of the existence of God provided [MASK] Anselm and Descartes , among others ? [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
on  |  on
straits  |  trouble
Paragraph: 
[CLS] Indeed , if the color teal [MASK] a team 's uniforms is any indication of the future , the Marlins are in dire [MASK] ! Refer to the San Jose Sharks for proof ... But I have hope for the Marlins . I was a sometime member of the Rene Lachemann fan club at the Oakland Colise

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (837 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
it  |  it
ivy  |  IVY
Paragraph: 
[CLS] Golly , I love stupid people . : - ) Listen , Rex , this is hockey . The NHL , to be precise . And in the NHL , there exist these things called `` ties '' . A tie occurs when a game ends with the score for each team equal . Each team gets one point for a tie . There also exits these things called `` wins '' . A win is when one team has a higher score than the opponent . ( Oh yeah , only two teams play each other at a time , so I can say `` the opponent '' . ) A team gets two points for a win . So , let 's say that a team has a record of 38 wins , 36 losses , and 10 ties . Another team has a record of 40 wins , 38 losses and 6 ties . The first team has ( 38*2 ) +10 = 86 points . The second team has ( 40*2 ) +6 = 86 points . WOW ! They *both* have the same number of points , but the number of wins is different ! How did they do that ? ? ! ? ! ? ! ? ! That 's amazing . S

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (813 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
but  |  long
can  |  can
Paragraph: 
[CLS] Well , there is a fair amount of evidence floating around that indicates that OTO has been around since at least the late 1800s , [MASK] before Crowley ever heard of it , how long has AMORC been around ? ( yes , I know that they claim to have existed as an organization clear into prehistory , but I doubt that they have any organizational paperwork as a non-profit that [MASK] be carbon-dated to 20,000 BC ) A.Lizard [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
listening  |  Connecting
side  |  back
Paragraph: 
[CLS] Note that if you get the external CD300 for your Centris or Q800 you will miss out on the sound mixing feature unless you are willing to run a wire from the motherboard sound input connector to the stereo output on the CD . [MASK] to the sound input port on the [MASK] 

In [9]:
def predict2(text, correct_word_list, tokenizer, model):
    
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    masked_list = []
    for i in range(len(tokenized_text)):
        if tokenized_text[i] == "[MASK]":
            masked_list.append(i)
            
    segments_ids = [0] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
        
#     prediction_test = predictions.clone().detach()
    
    predict_list=[]
    for i in masked_list:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        predict_list.append(predicted_token)
        
    for i in range(len(predict_list)):
        if predict_list[i].lower() != correct_word_list[i].lower():
            for j in range(len(predictions[0,masked_list[i]])):
                predicted_index = torch.argmax(predictions[0, masked_list[i]]).item()
                predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

                if correct_word_list[i][-2:].lower() not in predicted_token:
                    predictions[0,masked_list[i],int(torch.argmax(predictions[0,masked_list[i]]))] = 0
                    continue
                else:
                    predict_list[i] =  predicted_token
                    break
        
    print("----------------------------------------------")
    print("Predicted word | Original word: ")
    for i in range(len(correct_word_list)):
        print(predict_list[i]," | ",correct_word_list[i])
    print("Paragraph: ")
    print(text)
    print("----------------------------------------------")
        
    return predict_list

In [15]:
def model_test2(text_dict,tokenizer, model):
    
    total_num = 0
    correct_num = 0
    for text, word in text_dict.items():
        total_num = total_num+len(word)
        try:
            predict_list = predict2(text,word,tokenizer,model)
        except:
            continue
        for i in range(len(word)):
                if word[i].lower() == predict_list[i].lower():
                    correct_num+=1
                
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    print("The number of total masked word: ",total_num)
    print("The number of predicted word correctly: ",correct_num)
    print("Accuracy rate: ",correct_num/total_num)
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    
    return 

In [16]:
model_test2(dic,tokenizer,model)

----------------------------------------------
Predicted word | Original word: 
disappointed  |  disappointed
insanity  |  PENS
Paragraph: 
[CLS] I am sure some bashers of Pens fans are pretty confused about the lack of any kind of posts about the recent Pens massacre of the Devils . Actually , I am bit puzzled too and a bit relieved . However , I am going to put an end to non-PIttsburghers ' relief with a bit of praise for the Pens . Man , they are killing those Devils worse than I thought . Jagr just showed you why he is much better than his regular season stats . He is also a lot fo fun to watch in the playoffs . Bowman should let JAgr have a lot of fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway . I was very [MASK] not to see the Islanders lose the final regular season game . [MASK] RULE ! ! ! [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
ram  |  RAM

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1580 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
has  |  has
take  |  take
Paragraph: 
[CLS] How about Kirlian imaging ? I believe the FAQ for sci.skeptics ( sp ? ) [MASK] a nice write-up on this . They would certainly be most supportive on helping you to build such a device and connect to a 120Kvolt supply so that you can [MASK] a serious look at your `` aura '' ... : - ) [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
is  |  is
no  |  no
Paragraph: 
[CLS] There [MASK] [MASK] notion of heliocentric , or even galacticentric either . -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
rejected  |  criticized
some  |  some
Paragraph: 
[CLS] In the following report : _Turkey Ey

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (998 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
doubts  |  doubts
is  |  is
Paragraph: 
[CLS] ed > 1 . All of us that argue about gyroscopes , etc. , throughly understand ed > the technique of countersteering . me > Including all the ones who think that they countersteer all the way me > through a corner ? ? ed > Well ... all the way through a decreasing-radius corner , anyway ... Maybe they are riding around an ever-decreasing circle of lies which eventually leads to the truth ... . me > The official line here ( though I do have my [MASK] about it ) is that the me > front brake is applied first , followed by the rear brake , the idea being me > that you avoid locking up the rear after weight transfer takes place . Me too , though unfortunately the `` Official Line '' [MASK] the one that you have to adhere to if you want to get a full licence . The examiner 's guidelines are laid down by the government , and the basic rider education courses have no choi

...

----------------------------------------------
Predicted word | Original word: 
one  |  one
by  |  by
Paragraph: 
[CLS] May I ask why they are afraid to do so ? -- - Speaking of proofs of God , the funniest [MASK] I have ever seen was in a term paper handed in by a freshman . She wrote , `` God must exist , because he would n't be so mean as to make me believe he exists if he really does n't ! '' Is this argument really so much worse than the ontological proofs of the existence of God provided [MASK] Anselm and Descartes , among others ? [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
on  |  on
trouble  |  trouble
Paragraph: 
[CLS] Indeed , if the color teal [MASK] a team 's uniforms is any indication of the future , the Marlins are in dire [MASK] ! Refer to the San Jose Sharks for proof ... But I have hope for the Marlins . I was a sometime member of the Rene Lachemann fan club at the Oakland Coliseum

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (837 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
it  |  it
ivy  |  IVY
Paragraph: 
[CLS] Golly , I love stupid people . : - ) Listen , Rex , this is hockey . The NHL , to be precise . And in the NHL , there exist these things called `` ties '' . A tie occurs when a game ends with the score for each team equal . Each team gets one point for a tie . There also exits these things called `` wins '' . A win is when one team has a higher score than the opponent . ( Oh yeah , only two teams play each other at a time , so I can say `` the opponent '' . ) A team gets two points for a win . So , let 's say that a team has a record of 38 wins , 36 losses , and 10 ties . Another team has a record of 40 wins , 38 losses and 6 ties . The first team has ( 38*2 ) +10 = 86 points . The second team has ( 40*2 ) +6 = 86 points . WOW ! They *both* have the same number of points , but the number of wins is different ! How did they do that ? ? ! ? ! ? ! ? ! That 's amazing . S

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (813 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
long  |  long
can  |  can
Paragraph: 
[CLS] Well , there is a fair amount of evidence floating around that indicates that OTO has been around since at least the late 1800s , [MASK] before Crowley ever heard of it , how long has AMORC been around ? ( yes , I know that they claim to have existed as an organization clear into prehistory , but I doubt that they have any organizational paperwork as a non-profit that [MASK] be carbon-dated to 20,000 BC ) A.Lizard [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
listening  |  Connecting
back  |  back
Paragraph: 
[CLS] Note that if you get the external CD300 for your Centris or Q800 you will miss out on the sound mixing feature unless you are willing to run a wire from the motherboard sound input connector to the stereo output on the CD . [MASK] to the sound input port on the [MASK]

In [8]:
def predict3(text, correct_word_list, tokenizer, model):
    
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    masked_list = []
    for i in range(len(tokenized_text)):
        if tokenized_text[i] == "[MASK]":
            masked_list.append(i)
            
    segments_ids = [0] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
        
#     prediction_test = predictions.clone().detach()
    
    predict_list=[]
    for i in masked_list:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        predict_list.append(predicted_token)
        
    for i in range(1, len(predict_list)):
        if len(correct_word_list[i]) <= 3:
            continue
        else:
            let_loc = random_int_list(0,len(correct_word_list[i])-1,3)
            let_loc.sort()
            for j in range(len(predictions[0,masked_list[i]])):
                predicted_index = torch.argmax(predictions[0, masked_list[i]]).item()
                predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
                
                try:
                    if correct_word_list[i][let_loc[0]] != predicted_token[let_loc[0]] or correct_word_list[i][let_loc[1]] != predicted_token[let_loc[1]] or correct_word_list[i][let_loc[2]] != predicted_token[let_loc[2]]:
                        predictions[0,masked_list[i],int(torch.argmax(predictions[0,masked_list[i]]))] = 0
                        continue
                    else:
                        predict_list[i] =  predicted_token
                        break
                except:
                    predictions[0,masked_list[i],int(torch.argmax(predictions[0,masked_list[i]]))] = 0
                    continue
    print("----------------------------------------------")
    print("Predicted word | Original word: ")
    for i in range(len(correct_word_list)):
        print(predict_list[i]," | ",correct_word_list[i])
    print("Paragraph: ")
    print(text)
    print("----------------------------------------------")
        
    return predict_list

In [9]:
def model_test3(text_dict,tokenizer, model):
    
    total_num = 0
    correct_num = 0
    for text, word in text_dict.items():
        total_num = total_num+len(word)
        try:
            predict_list = predict3(text,word,tokenizer,model)
        except:
            continue
        for i in range(len(word)):
                if word[i].lower() == predict_list[i].lower():
                    correct_num+=1
                
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    print("The number of total masked word: ",total_num)
    print("The number of predicted word correctly: ",correct_num)
    print("Accuracy rate: ",correct_num/total_num)
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    
    return 

In [38]:
def get_letter_loc(word,ratio):
    """
    word length must be larger than 3
    """
    from math import ceil
    let_leng = ceil(ratio*len(word))
    loc = random_int_list(0,len(word)-1,let_leng)
    loc.sort()
    return loc

In [10]:
model_test3(dic,tokenizer,model)

----------------------------------------------
Predicted word | Original word: 
lucky  |  disappointed
team  |  PENS
Paragraph: 
[CLS] I am sure some bashers of Pens fans are pretty confused about the lack of any kind of posts about the recent Pens massacre of the Devils . Actually , I am bit puzzled too and a bit relieved . However , I am going to put an end to non-PIttsburghers ' relief with a bit of praise for the Pens . Man , they are killing those Devils worse than I thought . Jagr just showed you why he is much better than his regular season stats . He is also a lot fo fun to watch in the playoffs . Bowman should let JAgr have a lot of fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway . I was very [MASK] not to see the Islanders lose the final regular season game . [MASK] RULE ! ! ! [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
ram  |  RAM
graphics  

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1580 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
has  |  has
take  |  take
Paragraph: 
[CLS] How about Kirlian imaging ? I believe the FAQ for sci.skeptics ( sp ? ) [MASK] a nice write-up on this . They would certainly be most supportive on helping you to build such a device and connect to a 120Kvolt supply so that you can [MASK] a serious look at your `` aura '' ... : - ) [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
is  |  is
no  |  no
Paragraph: 
[CLS] There [MASK] [MASK] notion of heliocentric , or even galacticentric either . -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
rejects  |  criticized
some  |  some
Paragraph: 
[CLS] In the following report : _Turkey Eye

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (998 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
doubts  |  doubts
is  |  is
Paragraph: 
[CLS] ed > 1 . All of us that argue about gyroscopes , etc. , throughly understand ed > the technique of countersteering . me > Including all the ones who think that they countersteer all the way me > through a corner ? ? ed > Well ... all the way through a decreasing-radius corner , anyway ... Maybe they are riding around an ever-decreasing circle of lies which eventually leads to the truth ... . me > The official line here ( though I do have my [MASK] about it ) is that the me > front brake is applied first , followed by the rear brake , the idea being me > that you avoid locking up the rear after weight transfer takes place . Me too , though unfortunately the `` Official Line '' [MASK] the one that you have to adhere to if you want to get a full licence . The examiner 's guidelines are laid down by the government , and the basic rider education courses have no choi

...

----------------------------------------------
Predicted word | Original word: 
thing  |  one
by  |  by
Paragraph: 
[CLS] May I ask why they are afraid to do so ? -- - Speaking of proofs of God , the funniest [MASK] I have ever seen was in a term paper handed in by a freshman . She wrote , `` God must exist , because he would n't be so mean as to make me believe he exists if he really does n't ! '' Is this argument really so much worse than the ontological proofs of the existence of God provided [MASK] Anselm and Descartes , among others ? [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
on  |  on
trouble  |  trouble
Paragraph: 
[CLS] Indeed , if the color teal [MASK] a team 's uniforms is any indication of the future , the Marlins are in dire [MASK] ! Refer to the San Jose Sharks for proof ... But I have hope for the Marlins . I was a sometime member of the Rene Lachemann fan club at the Oakland Colise

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (837 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
it  |  it
ivy  |  IVY
Paragraph: 
[CLS] Golly , I love stupid people . : - ) Listen , Rex , this is hockey . The NHL , to be precise . And in the NHL , there exist these things called `` ties '' . A tie occurs when a game ends with the score for each team equal . Each team gets one point for a tie . There also exits these things called `` wins '' . A win is when one team has a higher score than the opponent . ( Oh yeah , only two teams play each other at a time , so I can say `` the opponent '' . ) A team gets two points for a win . So , let 's say that a team has a record of 38 wins , 36 losses , and 10 ties . Another team has a record of 40 wins , 38 losses and 6 ties . The first team has ( 38*2 ) +10 = 86 points . The second team has ( 40*2 ) +6 = 86 points . WOW ! They *both* have the same number of points , but the number of wins is different ! How did they do that ? ? ! ? ! ? ! ? ! That 's amazing . S

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (813 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
but  |  long
can  |  can
Paragraph: 
[CLS] Well , there is a fair amount of evidence floating around that indicates that OTO has been around since at least the late 1800s , [MASK] before Crowley ever heard of it , how long has AMORC been around ? ( yes , I know that they claim to have existed as an organization clear into prehistory , but I doubt that they have any organizational paperwork as a non-profit that [MASK] be carbon-dated to 20,000 BC ) A.Lizard [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
listening  |  Connecting
back  |  back
Paragraph: 
[CLS] Note that if you get the external CD300 for your Centris or Q800 you will miss out on the sound mixing feature unless you are willing to run a wire from the motherboard sound input connector to the stereo output on the CD . [MASK] to the sound input port on the [MASK] 